In [1]:
from z3 import *

# Horários Semanais

Há $P$ projetos, enumerados de $(0, ..., P-1)$. 

Cada projeto $p$ tem $C_p$ colaboradores $(0, ..., C_p-1)$ dos quais um é líder.

Para cada projeto $p$ há $N_p$ reuniões semanais.

As reuniões têm de conter sempre o líder e pelo menos $50\%$ dos colaboradores.

As reuniões ocorrem em uma das $S$ salas enumeradas de $(0, ..., S-1)$.

Os tempos são representados por slots $(\mathit{hora}, \mathit{dia})$.

In [5]:
class Schedule():
    def __init__(self, projects, rooms):
        self.rooms = rooms
        self.x_in = projects
        self.num_meetings = self.projects_to_min_meets()
        self.x_in = self.projects_to_tensor()
        
    def projects_to_min_meets(self):
        tensor = {}
        for p in self.x_in:
            tensor[p.project_id] = p.num_meetings
        return tensor
        
    def projects_to_tensor(self):
        tensor = {}
        for p in self.x_in:
            tensor[p.project_id] = p.collaborators
        return tensor


class Project():
    def __init__(self, project_id, num_meetings, leader, participants):
        self.project_id = project_id
        self.num_meetings = num_meetings
        self.collaborators = [leader] + participants
        self.collaborators = self.slots_to_tensor()
        
    def slots_to_tensor(self):
        days = ["mon", "tue", "wed", "thu", "fri"]
        tensor = {}
        for i in range(len(self.collaborators)):
            tensor[i] = {}
            for day in days:
                tensor[i][day] = {}
                for hour in range(8, 17):
                    tensor[i][day][hour] = 0
                    for (d, hs, he) in self.collaborators[i]:
                        if d == day and hour >= hs and hour < he:
                            tensor[i][day][hour] = 1
        return tensor

## Variáveis do problema

$x_{p, c, d, h}$: inteiro que representa a disponibilidade do colaborador $c$ do projeto $p$, onde $c=0$ é o líder, $d$ é o dia e $h$ a hora desse dia

$y_{s, d, h, p}$: inteiro que representa a disponibilidade da sala $s$ no dia $d$ e hora $h$ para o projeto $p$.


$r_{p, d, h, s}:$ inteiro que representa a existência de uma reunião para o projeto $p$ no dia $d$, hora $h$ e sala $s$.

## Condições inerentes ao problema

1. Todos os valores são inteiros unitários ou nulos:
$$ 0 \le x_{p,c,d,h} \le 1, \forall_{p,c,d,h} $$
$$ 0 \le y_{s,d,h,p} \le 1, \forall_{s,d,h,p} $$
$$ 0 \le r_{p,d,h,s} \le 1, \forall_{p,d,h,s} $$

2. Cada sala, por dia e hora, está disponível para reunião de apenas um projeto:
$$ \sum_{p=0}^{P-1} y_{s,d,h,p} = 1, \forall_{s, d, h} $$

3. Cada projeto só pode ter no máximo uma reunião por dia e hora:
$$ \sum_{s=0}^{S-1} r_{p,d,h,s} \le 1, \forall_{p,d,h} $$

4. Para haver reunião de um projeto numa certa sala, dia e hora, o líder e pelo menos $50\%$ dos colaboradores devem estar disponíveis e a sala também:
$$ r_{p,d,h,s} = 1 \rightarrow \left( y_{s,d,h,p} = 1 \wedge x_{p,0,d,h} = 1 \wedge \sum_{c=1}^{C_p-1} x_{p,c,d,h} \ge \frac{C_p-1}{2} \right), \forall_{p,d,h,s} $$

## Condições de input

1. A disponibilidade dos colaboradores de cada projeto $x_{p,c,d,h}^{(i)}$ é dada como input do problema:
$$ x_{p,c,d,h} = x_{p,c,d,h}^{(i)}, \forall_{p,c,d,h} $$

2. Deve haver um número mínimo de reuniões semanais por projeto $N_p$, dada como input do problema:
$$ \sum_{d=0}^{D-1} \sum_{h=h_i}^{h_f} \sum_{s=0}^{S-1} r_{p,s,d,h} \ge N_p, \forall_p $$

## Condição opcional

1. Se possível, marcar no máximo uma reunião por dia:
$$ \sum_{h=h_i}^{h_f} \sum_{s=0}^{S-1} r_{p,d,h,s} \le 1, \forall_{p,d} $$

In [11]:
def get_meetings(self):
    # Variáveis auxiliares
    days = ["mon", "tue", "wed", "thu", "fri"]
    rooms = self.rooms
    x_in = self.x_in

    solver = Solver()
    # Criar os dicionários de inteiros x, y e r descritos anteriormente
    x = {p: {c: {d: {h: Int(f'x:{p},{c},{d},{h}') for h in range(8, 17)} for d in days} for c in x_in[p]} for p in x_in}
    y = {s: {d: {h: {p: Int(f'y:{s},{d},{h},{p}') for p in x_in} for h in range(8, 17)} for d in days} for s in rooms}
    r = {p: {d: {h: {s: Int(f'r:{p},{s},{d},{h}') for s in rooms} for h in range(8, 17)} for d in days} for p in x_in}

    # Condição inerente 1 e de input 1
    for p in x:
        for c in x[p]:
            for d in x[p][c]:
                for h in x[p][c][d]:
                    solver.add(x[p][c][d][h] >= 0, x[p][c][d][h] <= 1)
                    solver.add(x[p][c][d][h] == x_in[p][c][d][h])
    for s in y:
        for d in y[s]:
            for h in y[s][d]:
                for p in y[s][d][h]:
                    solver.add(y[s][d][h][p] <= 1, y[s][d][h][p] >= 0)
                    solver.add(r[p][d][h][s] <= 1, r[p][d][h][s] >= 0)

    # Condição inerente 2
    for s in y:
        for d in y[s]:
            for h in y[s][d]:
                solver.add(sum([y[s][d][h][p] for p in y[s][d][h]]) == 1)
                
    # Condição inerente 3
    for p in r:
        for d in r[p]:
            for h in r[p][d]:
                solver.add(sum([r[p][d][h][s] for s in r[p][d][h]]) <= 1)

    # Condição inerente 4
    for p in r:
        C = len(x_in[p])
        for d in r[p]:
            for h in r[p][d]:
                for s in r[p][d][h]:
                    solver.add(Implies(r[p][d][h][s] == 1, x[p][0][d][h] >= 1))
                    solver.add(Implies(r[p][d][h][s] == 1, y[s][d][h][p] >= 1))
                    solver.add(Implies(r[p][d][h][s] == 1, sum([x[p][c][d][h] for c in range(1, C)]) >= (C-1)/2))
                    
    # Condição de input 2
    for p in r:
        N = self.num_meetings[p]
        soma = 0
        for d in r[p]:
            for h in r[p][d]:
                for s in r[p][d][h]:
                    soma += r[p][d][h][s]
        solver.add(soma >= N)
        
    # Condição opcional de uma reunião máxima por dia
    solver.push()
    for p in r:
        for d in r[p]:
            soma = 0
            for h in r[p][d]:
                for s in r[p][d][h]:
                    soma += r[p][d][h][s]
            solver.add(soma <= 1)
        
    if solver.check() == sat:
        print("This shedule is solvable.")
        return solver.model()
    else:
        # Condição opcional não verificada
        s.pop()
        if solver.check() == sat:
            print("This shedule is solvable with more than one meeting a day.")
            return solver.model()
        else:
            # Horário insatisfazível
            print("This schedule is not solvable!")
            return None
                    
Schedule.get_meetings = get_meetings

In [9]:
# Projeto 1
leader_1 = [("mon", 9, 11), ("tue", 10, 12), ("wed", 13, 15)]
participants_1 = [
    [("mon", 10, 13), ("tue", 8, 11), ("wed", 11, 14)],
    [("mon", 9, 10), ("wed", 13, 16), ("fri", 9, 13)]
]

# Projeto 2
leader_2 = [("wed", 9, 11), ("thu", 10, 12), ("fri", 13, 15)]
participants_2 = [
    [("mon", 10, 13), ("wed", 10, 12), ("thu", 11, 14)],
    [("tue", 9, 10), ("wed", 9, 12), ("fri", 12, 14)]
]

# Criar schedule
rooms = ["S_Fis", "S_Inf"]
startup = Schedule([Project("LC", 3, leader_1, participants_1), Project("InfQ", 2, leader_2, participants_2)], rooms)

In [12]:
r = startup.get_meetings()
r = [tuple(str(elem).split(',')) for elem in r if "r:" in str(elem) and r[elem] == 1]
r = sorted([(r1.split(':')[1], r3, r4, r2) for (r1, r2, r3, r4) in r])

This shedule is solvable.


In [13]:
print(r)

[('InfQ', 'thu', '11', 'S_Fis'), ('InfQ', 'wed', '9', 'S_Fis'), ('LC', 'mon', '9', 'S_Fis'), ('LC', 'tue', '10', 'S_Fis'), ('LC', 'wed', '13', 'S_Fis')]
